In [6]:
import time
import urllib2
import datetime
from itertools import ifilter
from collections import Counter, defaultdict
import xml.etree.ElementTree as ET

from bs4 import BeautifulSoup
import matplotlib.pylab as plt
import pandas as pd
import numpy as np
from pandas import HDFStore
import ads
from geopy.geocoders import Nominatim

%matplotlib inline

In [83]:
OAI = "{http://www.openarchives.org/OAI/2.0/}"
ARXIV = "{http://arxiv.org/OAI/arXiv/}"

df = pd.DataFrame(columns=("id", "created"))
base_url = "http://export.arxiv.org/oai2?verb=ListRecords&"
url = (base_url +
       "from=2015-01-01&until=2015-12-31&" +
       "metadataPrefix=arXiv&set=physics:astro-ph")

while True:
    print "fetching", url
    try:
        response = urllib2.urlopen(url)

    except urllib2.HTTPError, e:
        if e.code == 503:
            to = 30
            print "Got 503. Retrying after {0:d} seconds.".format(to)

            time.sleep(to)
            continue

        else:
            raise

    xml = response.read()

    root = ET.fromstring(xml)

    for record in root.find(OAI+'ListRecords').findall(OAI+"record"):
        arxiv_id = record.find(OAI+'header').find(OAI+'identifier')
        meta = record.find(OAI+'metadata')
        info = meta.find(ARXIV+"arXiv")
        created = info.find(ARXIV+"created").text
        created = datetime.datetime.strptime(created, "%Y-%m-%d")
        
        contents = {
                    'id': info.find(ARXIV+"id").text,
                    'created': created,
                    }

        df = df.append(contents, ignore_index=True)

    token = root.find(OAI+'ListRecords').find(OAI+"resumptionToken")
    if token is None or token.text is None:
        break

    else:
        url = base_url + "resumptionToken=%s"%(token.text)


fetching http://export.arxiv.org/oai2?verb=ListRecords&from=2015-01-01&until=2015-12-31&metadataPrefix=arXiv&set=physics:astro-ph
fetching http://export.arxiv.org/oai2?verb=ListRecords&resumptionToken=1043277|1001
Got 503. Retrying after 30 seconds.
fetching http://export.arxiv.org/oai2?verb=ListRecords&resumptionToken=1043277|1001
fetching http://export.arxiv.org/oai2?verb=ListRecords&resumptionToken=1043277|2001
Got 503. Retrying after 30 seconds.
fetching http://export.arxiv.org/oai2?verb=ListRecords&resumptionToken=1043277|2001
fetching http://export.arxiv.org/oai2?verb=ListRecords&resumptionToken=1043277|3001
Got 503. Retrying after 30 seconds.
fetching http://export.arxiv.org/oai2?verb=ListRecords&resumptionToken=1043277|3001
fetching http://export.arxiv.org/oai2?verb=ListRecords&resumptionToken=1043277|4001
Got 503. Retrying after 30 seconds.
fetching http://export.arxiv.org/oai2?verb=ListRecords&resumptionToken=1043277|4001
fetching http://export.arxiv.org/oai2?verb=ListRecords

In [127]:
df = df[df['created'].dt.year == 2015]

In [88]:
save = pd.HDFStore("/Users/harshilkamdar/a-year-of-astronomy-arxiv.h5")
save.close()

In [5]:
df = pd.read_hdf('/Users/harshilkamdar/a-year-of-astronomy-arxiv.h5')

TypeError: read_hdf() takes exactly 2 arguments (1 given)

In [121]:

data = {
    "id": list(df['id'].values),
    "date": list(df['created'].values),
    "locations": []
}

In [137]:
for i, arxiv_id in df['id'].iteritems():
    query = list(ads.SearchQuery(q='arXiv: %s' % df['id'][i]))
    data["locations"].append(query[0].aff)

APIResponseError: u'Rate limit was exceeded'

In [138]:
print(len(data["locations"]))

2464
